In [ ]:
import math
import pandas_datareader as web
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Dense, LSTM
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.data.experimental import AUTOTUNE as AUTO

In [ ]:
data_csv = pd.read_csv('../input/eur-usd-forex-pair-historical-data-2002-2019/eurusd_hour.csv')

In [ ]:
data_csv.head()

In [ ]:
data_csv.set_index(['Date'], inplace=True)

In [ ]:
data = data_csv.filter(['BC'])
data.head()

In [ ]:
plt.figure(figsize=(20,10))
plt.title('Close price history')
plt.plot(data['BC'])
plt.xlabel('Date')
plt.ylabel('Close price USD ($)')
plt.show()

In [ ]:
dataset = data.values

In [ ]:
training_data_len = math.ceil(len(dataset)*0.95)

In [ ]:
print('Lenght of the training dataset: {}'.format(training_data_len))

In [ ]:
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(dataset)

In [ ]:
scaled_data

In [ ]:
window = 30

In [ ]:
train_data = scaled_data[0:training_data_len,:]
x_train = []
y_train = []

for i in range(window, len(train_data)):
    x_train.append(train_data[i-window:i, 0])
    y_train.append(train_data[i, 0])

In [ ]:
print('The train dataset {}, and the labels {}'.format(len(x_train), len(y_train)))

In [ ]:
x_train, y_train = np.array(x_train), np.array(y_train)

In [ ]:
print('The shape of x_train: {}\nThe shape of y_train: {}' .format(x_train.shape, y_train.shape))

In [ ]:
x_train = np.reshape(x_train, newshape=(x_train.shape[0], x_train.shape[1], 1))

In [ ]:
print('The new shape of x_train: {}'.format(x_train.shape))

In [ ]:
def create_model():
    return Sequential(
    [LSTM(50, return_sequences=True, input_shape= (None, 1)),
    LSTM(50, return_sequences=False),
    Dense(100),
    Dense(50),
    Dense(1)])

In [ ]:
model = create_model()
model.compile(optimizer='adamax', loss='mean_squared_error')

In [ ]:
hist = model.fit(x_train, y_train, batch_size=30, epochs=50)

In [ ]:
window=30

In [ ]:
test_data = scaled_data[training_data_len-window: , :]
x_test = []
y_test = dataset[training_data_len:,:]
for i in range(window, len(test_data)):
      x_test.append(test_data[i-window:i,0])
  

In [ ]:
x_test = np.array(x_test)
x_test = np.reshape(x_test, newshape=(x_test.shape[0], x_test.shape[1], 1))
pred = model.predict(x_test)
pred = scaler.inverse_transform(pred)

In [ ]:
train = data[:training_data_len]
valid = data[training_data_len:]

valid['Predictions'] = pred

plt.figure(figsize=(20,10))
plt.title('Model')
plt.xlabel('Date')
plt.ylabel('Close_price_USD')
plt.plot(train['BC'])
plt.plot(valid[['BC', 'Predictions']])
plt.legend(['Train', 'Val','Pred'])
plt.show()

In [ ]:
valid.tail(5)

In [ ]:
train = data[:training_data_len]
valid = data[training_data_len:]

valid['Predictions'] = pred

plt.figure(figsize=(20,10))
plt.title('Model')
plt.xlabel('Date')
plt.ylabel('Close_price_USD')
plt.plot(valid[['BC', 'Predictions']])
plt.legend(['Val','Pred'])
plt.show()